# Wandecke

In [1]:
using MMJMesh
using MMJMesh.Plots
using MMJMesh.Meshes
using MMJMesh.Geometries
using MMJMesh.Mathematics

using SparseArrays
using DifferentialEquations
using CairoMakie: update_theme!, scatter!, lines, lines!, axislegend

include("fem.jl")
include("heat.jl")
update_theme!(colormap=:acton)

## Netz und Punkte

In [ ]:
m = Mesh("gmsh/corner.geo")

c1 = nodeindex(m, on(Point(0.000, 1.565)))
c2 = nodeindex(m, on(Point(0.115, 1.450)))
c3 = nodeindex(m, on(Point(0.315, 1.250)))
c4 = nodeindex(m, on(Point(0.565, 1.000)))
i1 = nodeindex(m, on(Point(1.565, 1.565)))
i2 = nodeindex(m, on(Point(1.565, 1.450)))
i3 = nodeindex(m, on(Point(1.565, 1.250)))
i4 = nodeindex(m, on(Point(1.565, 1.000)))
nds = [c1, c2, c3, c4, i1, i2, i3, i4]

p = mplot(m, edgesvisible=true) |> mconf()
scatter!(coordinates(m, nds))
p

## Berechnung

### Materialparameter und Randbedingungen

In [28]:
setdata!(group(m, :s1), :ke_func, heat_ke(1.0))
setdata!(group(m, :s1), :me_func, heat_me(1800, 1000))
setdata!(group(m, :s2), :ke_func, heat_ke(0.04))
setdata!(group(m, :s2), :me_func, heat_me(50, 900))
setdata!(group(m, :s3), :ke_func, heat_ke(2.3))
setdata!(group(m, :s3), :me_func, heat_me(2400, 1000))
setdata!(group(m, :wa), :ke_func, robin_ke(25))
setdata!(group(m, :wa), :re_func, robin_re(25, -5))
setdata!(group(m, :wi), :ke_func, robin_ke(7.5))
setdata!(group(m, :wi), :re_func, robin_re(7.5, 20))

### Systemmatrizen- und Vektor

In [29]:
K, M, r = assemble_kmr(m);

### Stationäre Berechnung

In [ ]:
theta0 = K \ r;
mplot(m, theta0) |> mconf()

## Transiente Berechnung

Anteile für innen und außen in `r` auf zwei Vektoren aufteilen

In [31]:
gi = group(m, :wi0)
ri = zeros(nnodes(m))
ri[gi] .= r[gi] 

ga = group(m, :wa0)
ra = zeros(nnodes(m))
ra[ga] .= r[ga];

Kontrolle

In [ ]:
K \ (ri + ra) ≈ theta0

Anfangswertproblem lösen

In [36]:
ω = 2 * π / (24 * 60 * 60)
F(T, _, t) = -K * T + ri + (1 + sin(ω * t)) * ra
JF(_, _, _) = -K

FM = ODEFunction(F, mass_matrix=M, jac=JF);
p = ODEProblem(FM, theta0, [0.0, 2 * 24 * 60 * 60])
sol = solve(p, QNDF());

Zeit und Temperaturen in Matrix

In [21]:
t = sol.t / 3600
theta = stack(sol.u, dims=1);

Plot außen

In [ ]:
p = lines(t, theta[:,c1], label="c1")
lines!(t, theta[:,c2], label="c2")
lines!(t, theta[:,i1], label="i1")
lines!(t, theta[:,i2], label="i2")
axislegend(position=:rb)
p

Plot innen

In [ ]:
p = lines(t, theta[:,c3], label="c3")
lines!(t, theta[:,c4], label="c4")
lines!(t, theta[:,i3], label="i3")
lines!(t, theta[:,i4], label="i4")
axislegend(position=:rb)
p

Temperaturverteilung im letzten Schritt

In [ ]:
mplot(m, sol[end]) |> mconf()